In [ ]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

## Initialization

### Imports

In [ ]:
import os
import sys
import json
import torch
import warnings
import numpy as np
import pandas as pd
import seaborn as sns

from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

sys.path.append("../code/")
# warnings.simplefilter("ignore", UserWarning)

In [ ]:
from params import *

from inference.main_test import k_fold_inf_test

from data.dataset import InferenceDataset
from data.transforms import HE_preprocess

### Inference

In [ ]:
log_folder = "../output/ext_aug_effb2_512_2_094/"  # 0.932

# log_folder = "../output/eff0_384_2_9401/"  # smaller model

In [ ]:
class Config:
    def __init__(self, **entries):
        self.__dict__.update(entries)

config = json.load(open(log_folder + 'config.json', 'r'))
config = Config(**config)

In [ ]:
# Fields not defined in old configs :
try:
    _ = config.tile_size
except:
    config.tile_size = 256
    config.reduce_factor = 4
    
try:
    _ = config.use_bot
except:
    config.use_bot = False
    config.use_fpn = False
    
try:
    _ = config.double_model
except:
    config.double_model = False

In [ ]:
df = pd.read_csv(DATA_PATH + "sample_submission.csv")

In [ ]:
config.overlap_factor = 1.5
use_tta = True
global_threshold = 0.4
config.selected_folds = [0, 1, 2, 3, 4]
save = True

In [ ]:
images = df['id'].values.tolist() + ["VAN0003-LK-32-21-PAS_registered.ome", "VAN0011-RK-3-10-PAS_registered.ome"]

In [ ]:
%%time
scores = k_fold_inf_test(
    config,
    images,
    log_folder=log_folder,
    use_full_size=False,
    global_threshold=global_threshold,
    use_tta=use_tta,
    save=save,
)